<a href="https://colab.research.google.com/github/sanjana-ak7/Bioinformatics_lab/blob/main/Bioinfo5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import sys

# --- Thermodynamic parameters (from kernel state) ---
# dH (enthalpy change, kcal/mol for dinucleotides)
dH = {'AA': -7.9, 'AT': -7.2, 'TA': -7.2, 'CA': -8.5, 'GT': -8.4, 'CT': -7.8, 'GA': -8.2, 'CG': -10.6, 'GC': -9.8, 'GG': -8.0, 'AG': -7.8, 'AC': -8.4, 'TG': -8.5, 'TC': -8.2}
# dS (entropy change, cal/mol/K for dinucleotides)
dS = {'AA': -22.2, 'AT': -20.4, 'TA': -21.3, 'CA': -22.7, 'GT': -22.4, 'CT': -21.0, 'GA': -22.2, 'CG': -27.2, 'GC': -24.4, 'GG': -19.9, 'AG': -21.0, 'AC': -22.4, 'TG': -22.7, 'TC': -22.2}
# Gas constant (cal/(mol*K))
R_GAS = 1.987

# --- Primer design criteria (from kernel state) ---
primer_length_min = 18
promer_length_max = 25
gc_content_min = 40
gc_content_max = 60
tm_min = 55
tm_max = 65
product_size_min = 100
product_size_max = 500

# --- Additional Constants for Tm calculation ---
salt_concentration = 0.05 # M (default for monovalent ions)
primer_concentration = 500e-9 # M (500 nM)

# --- Secondary structure check thresholds ---
# Max score for complementarity in self-dimer, cross-dimer, hairpin checks
# A score is generally the sum of G+C for complementary bases, or number of complementary bases.
# For simplicity, let's use the number of consecutive complementary bases.
MAX_DIMER_COMPLEMENTARITY = 5 # Max consecutive complementary bases allowed
MAX_HAIRPIN_COMPLEMENTARITY = 5 # Max consecutive complementary bases allowed for hairpin stem

# --- Helper Functions ---
def reverse_complement(seq):
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return "".join([complement.get(base, base) for base in seq[::-1].upper()])

def calculate_gc_content(seq):
    gc_count = seq.upper().count('G') + seq.upper().count('C')
    return (gc_count / len(seq)) * 100 if len(seq) > 0 else 0

def calculate_tm(primer_seq, dH_params, dS_params, R_gas, salt_conc, primer_conc):
    """Calculate melting temperature using nearest-neighbor thermodynamics."""
    primer_seq = primer_seq.upper()
    total_dH = 0
    total_dS = 0

    # Iterate through dinucleotides in the primer
    for i in range(len(primer_seq) - 1):
        dinucleotide = primer_seq[i : i+2]
        if dinucleotide in dH_params and dinucleotide in dS_params:
            total_dH += dH_params[dinucleotide] # kcal/mol
            total_dS += dS_params[dinucleotide] # cal/mol/K
        else:
            # Handle unknown dinucleotides, e.g., Ns or non-standard bases
            # For simplicity, if not in dict, assume 0 contribution or warn
            # print(f"Warning: Unknown dinucleotide {dinucleucleotide} in primer {primer_seq}", file=sys.stderr)
            pass

    # Convert dH to cal/mol for consistency with dS and R_gas
    total_dH_cal = total_dH * 1000

    # Initiation parameters (common approximations)
    # dH_init_AT = 2.2 * 1000 # cal/mol
    # dS_init_AT = 6.9 # cal/mol/K
    # dH_init_GC = 0
    # dS_init_GC = 0
    # Add initiation contributions. For simplicity, we can omit them for a basic nearest-neighbor sum,
    # or use general approximations. Given the provided dicts are only for dinucleotides,
    # a full initiation correction might be overcomplicating unless specific values are given.
    # Let's use simplified formula that implicitly handles some of this via the log term.

    # Symmetry correction: -1.4 cal/mol/K if sequence is self-complementary (palindromic)
    # For primers, this is generally not applied unless self-dimer is explicitly self-complementary
    # and one wants to calculate its Tm, which is beyond simple primer Tm calculation.

    # Formula: Tm = (total_dH_cal) / (total_dS + R_gas * ln(primer_conc / 4)) - 273.15 + 16.6 * log10(salt_conc)
    # Some sources use different constants for the log terms. Using a common simplified version for primers.
    # Often, a more empirical formula is used for primers, but we must use the provided dH/dS.
    try:
        # Simplified nearest-neighbor Tm without explicit initiation, relies on dinucleotide sums
        # Tm = 1000*dH_total / (dS_total + R*ln(C/4)) - 273.15 + 16.6*log10([Na+])
        # Primer concentration term in denominator is often for self-complementary oligos or duplexes.
        # For single strand annealing, a general simplification is sometimes used.
        # Let's use a widely accepted formula that includes the primer concentration term:
        Tm = (total_dH_cal) / (total_dS + R_gas * math.log(primer_conc / 4)) - 273.15 + 16.6 * math.log10(salt_conc)

    except ZeroDivisionError:
        return 0 # Should not happen with typical sequences
    except ValueError:
        return 0 # Log of non-positive number

    return Tm

def check_complementarity(seq1, seq2_rc, min_complementary_bases):
    """Checks for stretches of complementarity between seq1 and reverse complement of seq2_rc.
       Returns the maximum number of consecutive complementary bases found."""
    max_consecutive = 0
    for i in range(len(seq1) - min_complementary_bases + 1):
        for j in range(len(seq2_rc) - min_complementary_bases + 1):
            current_consecutive = 0
            for k in range(min_complementary_bases):
                if i + k < len(seq1) and j + k < len(seq2_rc) and seq1[i+k] == seq2_rc[j+k]:
                    current_consecutive += 1
                else:
                    break
            max_consecutive = max(max_consecutive, current_consecutive)
    return max_consecutive

def check_self_dimer(primer_seq, min_complementary_bases=MAX_DIMER_COMPLEMENTARITY):
    """Checks for self-dimerization by looking for complementarity within the primer itself (shifted).
       Returns max consecutive complementary bases."""
    rc_primer = reverse_complement(primer_seq)
    max_dimer_score = 0
    # Check complementarity of primer with its reverse complement (shifted)
    # Overlap can be full or partial
    # A simple way: check primer with rc_primer at various overlaps
    for offset in range(1, len(primer_seq)):
        # Forward strand part
        s1 = primer_seq[offset:]
        # Reverse complement strand part
        s2_rc = rc_primer[:len(primer_seq)-offset]
        max_dimer_score = max(max_dimer_score, check_complementarity(s1, s2_rc, min_complementary_bases))

        s1 = primer_seq[:len(primer_seq)-offset]
        s2_rc = rc_primer[offset:]
        max_dimer_score = max(max_dimer_score, check_complementarity(s1, s2_rc, min_complementary_bases))
    return max_dimer_score

def check_cross_dimer(primer1_seq, primer2_seq, min_complementary_bases=MAX_DIMER_COMPLEMENTARITY):
    """Checks for cross-dimerization between two primers.
       Returns max consecutive complementary bases."""
    primer2_rc = reverse_complement(primer2_seq)
    return check_complementarity(primer1_seq, primer2_rc, min_complementary_bases)

def check_hairpin(primer_seq, min_stem_bases=MAX_HAIRPIN_COMPLEMENTARITY, min_loop_size=3, max_loop_size=10):
    """Checks for hairpin formation within a primer.
       Returns max consecutive complementary bases in the stem."""
    max_hairpin_score = 0
    primer_rc = reverse_complement(primer_seq)
    n = len(primer_seq)

    for i in range(n - min_stem_bases - min_loop_size):
        for j in range(i + min_stem_bases + min_loop_size, n - min_stem_bases + 1):
            stem1_end = i + min_stem_bases
            loop_start = stem1_end
            loop_end = j
            stem2_start = j
            stem2_end = j + min_stem_bases

            # Ensure loop is within acceptable size and stem2 does not exceed primer length
            if (loop_end - loop_start >= min_loop_size and
                loop_end - loop_start <= max_loop_size and
                stem2_end <= n):

                stem1 = primer_seq[i : stem1_end]
                stem2_rc = primer_rc[n - stem2_end : n - stem2_start]

                # Check for complementarity between stem1 and reverse complement of stem2
                current_score = check_complementarity(stem1, stem2_rc, min_stem_bases)
                max_hairpin_score = max(max_hairpin_score, current_score)
    return max_hairpin_score

def is_valid_primer(primer_seq, dH, dS, R_GAS, salt_conc, primer_conc,
                   pl_min, pl_max, gc_min, gc_max, tm_min, tm_max):
    """Checks if a primer meets all individual validity criteria."""
    if not (pl_min <= len(primer_seq) <= pl_max):
        return False, "Length invalid"

    gc = calculate_gc_content(primer_seq)
    if not (gc_min <= gc <= gc_max):
        return False, "GC content invalid"

    tm = calculate_tm(primer_seq, dH, dS, R_GAS, salt_conc, primer_conc)
    if not (tm_min <= tm <= tm_max):
        return False, "Tm invalid"

    if check_self_dimer(primer_seq) >= MAX_DIMER_COMPLEMENTARITY:
        return False, "Self-dimer detected"

    if check_hairpin(primer_seq) >= MAX_HAIRPIN_COMPLEMENTARITY:
        return False, "Hairpin detected"

    return True, "Valid"

# --- Main primer design logic ---

# Try to use the uploaded sequence, otherwise use a sample sequence
seq = "GTTCGTTGCAACAAATTGATGAGCAATGCTTTTTTATAATGCCAACTTTGTACAAAAAAGTTGGCATGGCCCTGTGGATGCGCCTCCTGCCCCTGCTGGCGCTGCTGGCCCTCTGGGGACCTGACCCAGCCGCAGCCTTTGTGAACCAACACCTGTGCGGCTCACACCTGGTGGAAGCTCTCTACCTAGTGTGCGGGGAACGAGGCTTCTTCTACACACCCAAGACCCGCCGGGAGGCAGAGGACCTGCAGGTGGGGCAGGTGGAGCTGGGCGGGGGCCC TGTGCAGGCAGCCTGCAGCCCTTGGCCCTGGAGGGGTCCCTGCAGAAGCGTTGCATTGTGGAACAATGCTGTACCAGCATCTGCTCCCTCTACCAGCTGGAGAACTACTGCAACTACCCAACTTTCTTGTACAAAGTTGGCATTATAAGAAAGCATTGCTTATCAATTTGTTGCAACGAAC"
try:
    # Check if 'seq' from 'iR1JC90fcO6Z' is available in the current environment's global scope
    if 'seq' in globals() and seq:
        target_sequence = seq.upper() # Ensure uppercase
        print(f"Using uploaded DNA sequence of length: {len(target_sequence)}")
    else:
        raise NameError("'seq' variable not found or is empty")
except NameError:
    print("Warning: 'seq' variable not found or is empty. Using a sample DNA sequence for demonstration.")
    target_sequence = "GAATTCGCATGCTAGCTAGCATGTAGCATGCTAGCTAGCATGTAGCATGCCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGACGGATCCGTCGATGAATTC".upper()
    print(f"Using sample DNA sequence of length: {len(target_sequence)}")


forward_primers = []
reverse_primers = []

# Generate forward primer candidates
for i in range(len(target_sequence) - primer_length_max - product_size_min + 1):
    for length in range(primer_length_min, primer_length_max + 1):
        if i + length <= len(target_sequence):
            primer_candidate = target_sequence[i : i + length]
            is_valid, reason = is_valid_primer(primer_candidate, dH, dS, R_GAS, salt_concentration, primer_concentration,
                                              primer_length_min, primer_length_max, gc_content_min, gc_content_max, tm_min, tm_max)
            if is_valid:
                tm = calculate_tm(primer_candidate, dH, dS, R_GAS, salt_concentration, primer_concentration)
                forward_primers.append({'sequence': primer_candidate, 'start_pos': i, 'tm': tm, 'gc': calculate_gc_content(primer_candidate)})

# Generate reverse primer candidates (from reverse complement of template)
# Reverse primers bind to the reverse complement strand, so we search the template's reverse complement
# and then take the reverse complement of the found sequence to get the primer sequence itself.
rc_target_sequence = reverse_complement(target_sequence)

for i in range(len(rc_target_sequence) - primer_length_max - product_size_min + 1):
    for length in range(primer_length_min, primer_length_max + 1):
        if i + length <= len(rc_target_sequence):
            # This is the binding site on the reverse complement strand
            binding_site_rc = rc_target_sequence[i : i + length]
            # The actual primer sequence is the reverse complement of this binding site
            primer_candidate = reverse_complement(binding_site_rc)

            is_valid, reason = is_valid_primer(primer_candidate, dH, dS, R_GAS, salt_concentration, primer_concentration,
                                              primer_length_min, primer_length_max, gc_content_min, gc_content_max, tm_min, tm_max)
            if is_valid:
                tm = calculate_tm(primer_candidate, dH, dS, R_GAS, salt_concentration, primer_concentration)
                # Store original start position on the reverse complement for calculating product size later
                # This corresponds to end position on the forward strand: len(target_sequence) - i
                reverse_primers.append({'sequence': primer_candidate, 'start_pos_rc': i, 'tm': tm, 'gc': calculate_gc_content(primer_candidate)})

print(f"Found {len(forward_primers)} valid forward primers.")
print(f"Found {len(reverse_primers)} valid reverse primers.\n")

# Find optimal primer pairs
optimal_primer_pairs = []
MAX_TM_DIFFERENCE = 5 # max allowed Tm difference between forward and reverse primers

for fp in forward_primers:
    for rp in reverse_primers:
        # Calculate product size
        # fp['start_pos'] is 0-indexed start of forward primer on forward strand
        # rp['start_pos_rc'] is 0-indexed start of reverse primer on reverse complement strand
        # End position of reverse primer on forward strand is (len(target_sequence) - rp['start_pos_rc'])
        # Start position of reverse primer on forward strand is (len(target_sequence) - rp['start_pos_rc'] - len(rp['sequence']))

        # Product starts at fp['start_pos'] and ends at end of rp's binding site on forward strand.
        # The reverse primer sequence is given as 5' to 3', but it binds to the reverse complement
        # of the template. So, its 'start_pos_rc' is the start of its binding site on the RC template.
        # This means the *end* of the amplicon on the *forward* template strand will be at
        # len(target_sequence) - rp['start_pos_rc']
        # And the start of the amplicon on the *forward* template strand is fp['start_pos']
        # So, the product size is (end_of_rp_binding_on_forward - start_of_fp_binding_on_forward)

        rp_binding_end_on_forward = len(target_sequence) - rp['start_pos_rc']
        product_start = fp['start_pos']

        product_size = rp_binding_end_on_forward - product_start

        if not (product_size_min <= product_size <= product_size_max):
            continue

        # Check Tm difference
        tm_diff = abs(fp['tm'] - rp['tm'])
        if tm_diff > MAX_TM_DIFFERENCE:
            continue

        # Check cross-dimer
        if check_cross_dimer(fp['sequence'], rp['sequence']) >= MAX_DIMER_COMPLEMENTARITY:
            continue

        optimal_primer_pairs.append({
            'forward_primer': fp,
            'reverse_primer': rp,
            'product_size': product_size,
            'tm_difference': tm_diff
        })

# Sort optimal pairs (e.g., by Tm difference, then product size close to middle)
optimal_primer_pairs.sort(key=lambda x: (x['tm_difference'], abs(x['product_size'] - (product_size_min + product_size_max) / 2)))

print(f"Found {len(optimal_primer_pairs)} optimal primer pairs.\n")

# Display top 5 optimal primer pairs
print("Top 5 Optimal Primer Pairs:")
if not optimal_primer_pairs:
    print("No optimal primer pairs found given the criteria.")
for i, pair in enumerate(optimal_primer_pairs[:5]):
    print(f"--- Pair {i+1} ---")
    print(f"  Forward Primer: {pair['forward_primer']['sequence']}")
    print(f"    Start Pos: {pair['forward_primer']['start_pos']}")
    print(f"    Tm: {pair['forward_primer']['tm']:.2f} C")
    print(f"    GC Content: {pair['forward_primer']['gc']:.2f}%")
    print(f"  Reverse Primer: {pair['reverse_primer']['sequence']}")
    print(f"    Start Pos (on RC): {pair['reverse_primer']['start_pos_rc']}")
    print(f"    Tm: {pair['reverse_primer']['tm']:.2f} C")
    print(f"    GC Content: {pair['reverse_primer']['gc']:.2f}%")
    print(f"  Product Size: {pair['product_size']} bp")
    print(f"  Tm Difference: {pair['tm_difference']:.2f} C\n")

Using uploaded DNA sequence of length: 462
Found 183 valid forward primers.
Found 162 valid reverse primers.

Found 3861 optimal primer pairs.

Top 5 Optimal Primer Pairs:
--- Pair 1 ---
  Forward Primer: CTTTGTGAACCAACACCTGTGCGGC
    Start Pos: 136
    Tm: 57.94 C
    GC Content: 56.00%
  Reverse Primer: TGCAGAAGCGTTGCATTGTGGA
    Start Pos (on RC): 119
    Tm: 57.94 C
    GC Content: 50.00%
  Product Size: 207 bp
  Tm Difference: 0.00 C

--- Pair 2 ---
  Forward Primer: TTTGTGAACCAACACCTGTGCGGCT
    Start Pos: 137
    Tm: 57.94 C
    GC Content: 52.00%
  Reverse Primer: TGCAGAAGCGTTGCATTGTGGA
    Start Pos (on RC): 119
    Tm: 57.94 C
    GC Content: 50.00%
  Product Size: 206 bp
  Tm Difference: 0.00 C

--- Pair 3 ---
  Forward Primer: TTGTGAACCAACACCTGTGCGGCT
    Start Pos: 138
    Tm: 57.94 C
    GC Content: 54.17%
  Reverse Primer: TGCAGAAGCGTTGCATTGTGGA
    Start Pos (on RC): 119
    Tm: 57.94 C
    GC Content: 50.00%
  Product Size: 205 bp
  Tm Difference: 0.00 C

--- Pair 4 --